In [16]:
import numpy as np
from scipy.io import loadmat

# Load one patient's file
data = loadmat('Data/Temp Data/gz2.mat')

# Extract arrays (crop version is usually preprocessed & aligned)
left_plantar_list = data['Direct_plantar_Left_crop']
right_plantar_list = data['Direct_plantar_Right_crop']

# Convert from nested objects to numpy arrays
# each entry in the (10,1) list is an array (for 10 days)
left_plantar = [arr for arr in left_plantar_list[:,0]]
right_plantar = [arr for arr in right_plantar_list[:,0]]


The below function needs to be changed most important!!

In [17]:
def analyze_day(left_img, right_img, point_thresh=2.2, avg_thresh=1.6):
    """
    Compare left vs right thermal images for one day.
    """
    # Ensure shapes match
    assert left_img.shape == right_img.shape, "Left and Right images must be aligned"

    # Point-wise absolute difference
    diff = np.abs(left_img - right_img)

    # Rule 1: Point-wise asymmetry
    point_violation = np.any(diff > point_thresh)

    # Rule 2: Average asymmetry
    avg_diff = np.abs(np.mean(left_img) - np.mean(right_img))
    avg_violation = avg_diff > avg_thresh

    return {
        "point_violation": point_violation,
        "avg_violation": avg_violation,
        "avg_diff": avg_diff,
        "max_point_diff": np.max(diff)
    }


In [ ]:
def analyze_over_days(left_imgs, right_imgs, days_required=5):
    results = []
    for l_img, r_img in zip(left_imgs, right_imgs):
        results.append(analyze_day(l_img, r_img))

    # Temporal persistence: if violation persists for N+ days
    at_risk_days = [res["point_violation"] or res["avg_violation"] for res in results]
    
    # Check if there’s a consecutive streak of at least days_required
    risk_flag = False
    streak = 0
    for day_flag in at_risk_days:
        if day_flag:
            streak += 1
            if streak >= days_required:
                risk_flag = True
                break
        else:
            streak = 0

    return results, risk_flag


In [ ]:
results, risk_flag = analyze_over_days(left_plantar, right_plantar, days_required=5)

print("Daily results:")
for i, res in enumerate(results):
    print(f"Day {i+1}: AvgDiff={res['avg_diff']:.2f}, MaxPointDiff={res['max_point_diff']:.2f}, "
          f"PointFlag={res['point_violation']}, AvgFlag={res['avg_violation']}")

print("\nFinal Risk Flag:", risk_flag)


Daily results:
Day 1: AvgDiff=0.75, MaxPointDiff=33.30, PointFlag=True, AvgFlag=False
Day 2: AvgDiff=0.74, MaxPointDiff=32.70, PointFlag=True, AvgFlag=False
Day 3: AvgDiff=0.71, MaxPointDiff=31.60, PointFlag=True, AvgFlag=False
Day 4: AvgDiff=0.72, MaxPointDiff=31.80, PointFlag=True, AvgFlag=False
Day 5: AvgDiff=0.78, MaxPointDiff=33.10, PointFlag=True, AvgFlag=False
Day 6: AvgDiff=0.87, MaxPointDiff=34.30, PointFlag=True, AvgFlag=False
Day 7: AvgDiff=0.84, MaxPointDiff=34.20, PointFlag=True, AvgFlag=False
Day 8: AvgDiff=0.80, MaxPointDiff=34.30, PointFlag=True, AvgFlag=False
Day 9: AvgDiff=0.78, MaxPointDiff=32.90, PointFlag=True, AvgFlag=False
Day 10: AvgDiff=0.74, MaxPointDiff=31.50, PointFlag=True, AvgFlag=False

Final Risk Flag: True
